# Задание по SQL

Для проверки корректности выполнения задания сначала необходимо создать базу данных, содержащую таблицы Staff и Jobtitles.

Первым делом необходимо подключить библиотеку sqlite3.

In [ ]:
import sqlite3

Создаем БД

In [ ]:
# Подключаемся к базе данных (если базы данных нет, она будет создана)
conn = sqlite3.connect('example.db')

# Создаем объект-курсор для выполнения SQL-запросов
cursor = conn.cursor()

# Создаем таблицу Jobtitles
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Jobtitles (
        jobtitle_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    )
''')

# Заполняем таблицу Jobtitles данными
jobtitles_data = [
    (1, 'Разработчик'),
    (2, 'Системный аналитик'),
    (3, 'Менеджер проектов'),
    (4, 'Системный администратор'),
    (5, 'Руководитель группы'),
    (6, 'Инженер тестирования'),
    (7, 'Сотрудник группы поддержки')
]
cursor.executemany('INSERT INTO Jobtitles VALUES (?, ?)', jobtitles_data)

# Создаем таблицу Staff
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Staff (
        staff_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        salary REAL,
        email TEXT,
        birthday DATETIME,
        jobtitle_id INTEGER,
        FOREIGN KEY (jobtitle_id) REFERENCES Jobtitles(jobtitle_id)
    )
''')

# Заполняем таблицу Staff данными
staff_data = [
    (1, 'Иванов Сергей', 100000, 'test@test.ru', '1990-03-03', 1),
    (2, 'Петров Пётр', 60000, 'petr@test.ru', '2000-01-12', 7),
    (3, 'Сидоров Василий', 80000, 'test@test.ru', '1999-04-02', 6),
    (4, 'Максимов Иван', 70000, 'ivan.m@test.ru', '1997-02-10', 4),
    (5, 'Попов Иван', 120000, 'popov@test.ru', '2001-04-25', 5)
]
cursor.executemany('INSERT INTO Staff VALUES (?, ?, ?, ?, ?, ?)', staff_data)

# Подтверждаем изменения в базе данных
conn.commit()

***Задание 1.*** Напишите запрос, с помощью которого можно найти дубли в поле email из таблицы Sfaff.

In [ ]:
cursor.execute('''
    SELECT email
    FROM Staff
    GROUP BY email
    HAVING COUNT(email) > 1
''')
duplicates = cursor.fetchall()
print("Дубли в поле email:")
for row in duplicates:
    print(row[0])

Дубли в поле email:
test@test.ru


***Задание 2.*** Напишите запрос, с помощью которого можно определить возраст каждого сотрудника из таблицы Staff на момент запроса.

In [ ]:
cursor.execute('''
    SELECT name,
           (CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', birthday) AS INTEGER)) -
           (strftime('%m-%d', 'now') < strftime('%m-%d', birthday)) AS age
    FROM Staff
''')
ages = cursor.fetchall()
print("\nВозраст каждого сотрудника:")
for row in ages:
    print(row[0], "-", row[1])


Возраст каждого сотрудника:
Иванов Сергей - 33
Петров Пётр - 24
Сидоров Василий - 24
Максимов Иван - 27
Попов Иван - 22


***Задание 3.*** Напишите запрос, с помощью которого можно определить должность (Jobtitles.name) со вторым по величине уровнем зарплаты.

In [ ]:
cursor.execute('''
    SELECT jt.name
    FROM Jobtitles jt
    JOIN Staff s ON jt.jobtitle_id = s.jobtitle_id
    GROUP BY jt.name
    ORDER BY AVG(s.salary) DESC LIMIT 1 OFFSET 1
''')
second_highest_salary_jobtitle = cursor.fetchone()
print("\nДолжность со вторым по величине уровнем зарплаты:", second_highest_salary_jobtitle[0])


Должность со вторым по величине уровнем зарплаты: Разработчик


In [ ]:
# Закрываем соединение с базой данных
conn.close()